Send Feedback
Titanic dataset is one of the datasets available in sklearn.
You are given:
    1. A Training dataset csv file with X train and Y train data
    2. A X test File and you have to predict and submit predictions for this file.
Your task is to:
    1. Use Logistic Regression and come with predictions.
Read Instructions carefully -
    1. Use Logistic Regression as a training algorithm and submit results predicted.
    2. Files are in csv format.
    3. Submit a csv file with only predictions for X test data. File should not have any headers and should only have one column i.e. predictions.
    4. Your score is based on number of accurate predictions.

In [ ]:
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA


In [ ]:
# Loading Dataset
dataframe = pd.read_csv("training_titanic_x_y_train.csv", delimiter = ",")
dataframe.head()


In [ ]:
dataframe.describe()

In [ ]:
df = dataframe.copy()
sex = df['Sex'].values
sex_int = []
for s in sex :
    sex_int.append(1 if s == 'male' else 0)
df['Sex'] = sex_int
df.drop('Name', axis = 1, inplace = True)
df.drop('Ticket', axis = 1, inplace = True)
df.drop('Cabin', axis = 1, inplace = True)

embarked_counts = { i :(df['Embarked'] == i).sum() for i in set(df['Embarked'])}

def embarked_int(s):
    return embarked_counts[s]
df['Embarked'] = [embarked_int(i) for i in df['Embarked']]
#df.drop('Embarked', axis = 1, inplace = True)

ages = df['Age'].fillna(0).values
ages_nonzero = ages[np.nonzero(ages)]

mean_age = np.array(ages_nonzero).mean()


survived_female_ages = np.array(df['Survived'] * (1-df['Sex']) * ages )
indices_sfa = np.nonzero(survived_female_ages)
count_sfa = len(survived_female_ages[indices_sfa])
sum_sfa = np.sum(survived_female_ages)
mean_sfa = sum_sfa/count_sfa

not_survived_female_ages = np.array((1 - df['Survived']) * (1-df['Sex']) * ages )
indices_nsfa = np.nonzero(not_survived_female_ages)
count_nsfa = len(not_survived_female_ages[indices_nsfa])
sum_nsfa = np.sum(not_survived_female_ages)
mean_nsfa = sum_nsfa/count_nsfa

survived_male_ages = np.array(df['Survived'] * (df['Sex']) * ages )
indices_sma = np.nonzero(survived_male_ages)

count_sma = len(survived_male_ages[indices_sma])
sum_sma = np.sum(survived_male_ages)
mean_sma = sum_sma/count_sma

not_survived_male_ages = np.array((1 - df['Survived']) * (df['Sex']) * ages )
indices_nsma = np.nonzero(not_survived_male_ages)

count_nsma = len(not_survived_male_ages[indices_nsma])
sum_nsma = np.sum(not_survived_male_ages)
mean_nsma = sum_nsma/count_nsma

print(mean_sma, mean_sfa, mean_nsma, mean_nsfa)
df.describe()

In [ ]:
#print(training_final_df.shape)
#print(df.shape)
values = np.empty((0,8))
for row in df.values :
    #print(row[2])
    if np.isnan(row[2]) : 
        if row[1] == 1.0 : # male
            if row[-1] == 1.0: # survived 
                row[2] = mean_sma
            else:
                row[2] = mean_nsma
        else: # female
            if row[-1] == 1: # survived
                row[2] = mean_sfa
            else:
                row[2] = mean_nsfa
    values = np.insert(values,0, row, axis = 0) 
    #training_final_df.append(row)
training_final_df = pd.DataFrame(values, columns = df.columns)

print(training_final_df.describe())
#print(values)


In [ ]:
final_X_train = training_final_df.values
print(final_X_train.shape)
X_train = training_final_df.drop('Survived', axis = 1).values
Y_train = training_final_df['Survived'].values
print(X_train.shape)
print(Y_train.shape)


In [ ]:
df.fillna(mean_age, inplace = True)
print(df)
df.describe()

X_df = df.drop('Survived', axis = 1)
X_train = X_df.values
Y_train = df['Survived'].values

print("Shape of X_train ", X_train.shape)
print("Shape of Y_train ", Y_train.shape)

test_dataframe = pd.read_csv("test_titanic_x_test.csv", delimiter = ",")


test_df = test_dataframe.copy()
sex = test_df['Sex'].values
sex_int = []
for s in sex :
    sex_int.append(1 if s == 'male' else 0)
test_df['Sex'] = sex_int
test_df.drop('Name', axis = 1, inplace = True)
test_df.drop('Ticket', axis = 1, inplace = True)
test_df.drop('Cabin', axis = 1, inplace = True)

test_df['Embarked'] = [embarked_int(i) for i in test_df['Embarked']]

#df.drop('Embarked', axis = 1, inplace = True)

ages = test_df['Age'].fillna(0).values
ages_nonzero = ages[np.nonzero(ages)]

X_test = test_df.values

print("Shape of X_test ", X_test.shape)


In [ ]:
mean_fa = (mean_sfa * count_sfa + mean_nsfa * count_nsfa)/(count_nsfa + count_sfa)
mean_ma = (mean_sma * count_sma + mean_nsma * count_nsma)/(count_nsma + count_sma)

values = np.empty((0,7))
for row in test_df.values :
    #print(row[2])
    if np.isnan(row[2]) : 
        if row[1] == 1.0 : # male
            row[2] = mean_ma
        else: # female
            row[2] = mean_fa
    values = np.insert(values,0, row, axis = 0) 
    #training_final_df.append(row)
testing_final_df = pd.DataFrame(values, columns = test_df.columns)

print(testing_final_df.describe())
#print(values)


In [ ]:
X_test = testing_final_df.values
print(X_test.shape)

In [ ]:
import sklearn.preprocessing as pps

def add_more_features (X_train, imp_cols_indices = []):
    
    X_train_df = pd.DataFrame(X_train)
    num_f = len(X_train_df.columns)
    col_names = X_train_df.columns
    
    if len(imp_cols_indices) == 0 :
        imp_cols_indices = np.arange(num_f)
    new_df = X_train_df.copy()
    num_imp_cols = len(imp_cols_indices)
    
    pow = 3
    
    while pow < 10 :
        for i1 in range(num_imp_cols) :
            i = imp_cols_indices[i1]
            new_df[ str(col_names[i]) + "_pow_" + str(pow) ] = X_train_df[i] ** pow
        pow += 1
    
    for i1 in range(num_imp_cols) :
        for j1 in range(i1, num_imp_cols):
            i = imp_cols_indices[i1]
            j = imp_cols_indices[j1]
            new_df[ str(col_names[i]) + "_" + str(col_names[j])] = X_train_df[i] * X_train_df[j]
            new_df[ str(col_names[i])*2 + "_" + str(col_names[j])] = X_train_df[i]**2 * X_train_df[j]
            new_df[ str(col_names[i]) + "_" + str(col_names[j])*2 ] = X_train_df[i] * X_train_df[j]**2
            new_df[ str(col_names[i])*2 + "_" + str(col_names[j])*2 ] = X_train_df[i]**2 * X_train_df[j]**2
            
            new_df[ str(col_names[i])*3 + "_" + str(col_names[j])] = X_train_df[i]**3 * X_train_df[j]
            new_df[ str(col_names[i])*3 + "_" + str(col_names[j])*2] = X_train_df[i]**3 * X_train_df[j]**2
            new_df[ str(col_names[i])*3 + "_" + str(col_names[j])*3 ] = X_train_df[i]**3 * X_train_df[j]**3
            new_df[ str(col_names[i])*2 + "_" + str(col_names[j])*3 ] = X_train_df[i]**2 * X_train_df[j]**3
            new_df[ str(col_names[i]) + "_" + str(col_names[j])*3 ] = X_train_df[i] * X_train_df[j]**3
            
            
            
    print(new_df.describe())
    return new_df.values


X_train_new = add_more_features(X_train)
X_test_new = add_more_features(X_test)

std_scaler = pps.StandardScaler()
std_scaler.fit(X_train_new)

X_train_scaled = std_scaler.transform(X_train_new)
X_test_scaled = std_scaler.transform(X_test_new)

import copy
X_train_original = copy.deepcopy(X_train)
X_test_original = copy.deepcopy(X_test)

X_train = X_train_scaled
X_test = X_test_scaled

print(X_train.shape)
print(X_test.shape)


In [ ]:
X_t_df = pd.DataFrame(X_train)
X_t_df.describe()

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import model_selection

model = GradientBoostingClassifier(loss = 'exponential', subsample = 1, n_estimators = 100, max_depth =  8, warm_start = True, max_leaf_nodes=None, min_samples_leaf=1)
model.fit(X_train, Y_train)
print("Training Score: ", model.score(X_train, Y_train))
Y_pred = model.predict(X_test)


In [ ]:
print(Y_pred)

In [1]:
np.savetxt("y_pred.csv", Y_pred.astype(int), fmt = "%d") 

NameError: name 'np' is not defined